In [1]:
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']
df = pd.read_csv(url,names = columns)

In [2]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
import numpy as np

cols_with_missing_values = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_values] = df[cols_with_missing_values].replace(0,np.nan)
df.fillna(df.mean(),inplace= True)

print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [6]:
X = df.drop('Outcome',axis = 1)
y = df['Outcome']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators',50,100)
    max_depth = trial.suggest_int('max_depth',3,20)

    model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth)

    score = cross_val_score(model,X_train,y_train,cv=3,scoring='accuracy').mean()

    return score

study = optuna.create_study(direction = 'maximize', sampler = optuna.samplers.TPESampler())
study.optimize(objective,n_trials=50)

[I 2025-12-25 19:25:47,141] A new study created in memory with name: no-name-82224a86-a989-4e70-b392-7460d121b717
[I 2025-12-25 19:25:47,480] Trial 0 finished with value: 0.7345289335246293 and parameters: {'n_estimators': 90, 'max_depth': 20}. Best is trial 0 with value: 0.7345289335246293.
[I 2025-12-25 19:25:47,677] Trial 1 finished with value: 0.7606009883628247 and parameters: {'n_estimators': 59, 'max_depth': 10}. Best is trial 1 with value: 0.7606009883628247.
[I 2025-12-25 19:25:47,924] Trial 2 finished with value: 0.755738880918221 and parameters: {'n_estimators': 99, 'max_depth': 3}. Best is trial 1 with value: 0.7606009883628247.
[I 2025-12-25 19:25:48,232] Trial 3 finished with value: 0.7540969233221744 and parameters: {'n_estimators': 63, 'max_depth': 18}. Best is trial 1 with value: 0.7606009883628247.
[I 2025-12-25 19:25:48,619] Trial 4 finished with value: 0.7459588713534194 and parameters: {'n_estimators': 94, 'max_depth': 15}. Best is trial 1 with value: 0.76060098836

In [8]:
print(study.best_trial.value)
print(study.best_trial.params)

0.7622429459588713
{'n_estimators': 54, 'max_depth': 15}


In [9]:
from sklearn.metrics import accuracy_score

best_model = RandomForestClassifier(**study.best_trial.params)

best_model.fit(X_train,y_train)

y_pred = best_model.predict(X_test)

print(accuracy_score(y_test,y_pred))

0.7662337662337663


In [10]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-12-25 19:30:39,600] A new study created in memory with name: no-name-d8c152d5-afa9-4dff-a44b-38c50ce0bd99


Test Accuracy with best hyperparameters: 0.79


[I 2025-12-25 19:30:40,221] Trial 0 finished with value: 0.7459349593495935 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7459349593495935.
c:\work_repo\AI_ML_DS_GenAI_Portfolio\.myvenv\Lib\site-packages\optuna\samplers\_grid.py:199: UserWarning: The value `150` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntDistribution(high=100, log=False, low=50, step=1)`.
  warnings.warn(
[I 2025-12-25 19:30:41,509] Trial 1 finished with value: 0.747592858281524 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 1 with value: 0.747592858281524.
[I 2025-12-25 19:30:42,041] Trial 2 finished with value: 0.7427387215048621 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 1 with value: 0.747592858281524.
[I 2025-12-25 19:30:43,049] Trial 3 finished with value: 0.7654949784791966 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 3 with value: 0.765

In [11]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Best trial accuracy: 0.7654949784791966
Best hyperparameters: {'n_estimators': 100, 'max_depth': 15}
Test Accuracy with best hyperparameters: 0.77


In [12]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [13]:
# 1. Optimization History
plot_optimization_history(study).show()

In [14]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [15]:
# 3. Slice Plot
plot_slice(study).show()

In [16]:
# 4. Contour Plot
plot_contour(study).show()

In [17]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

ZeroDivisionError: Weights sum to zero, can't be normalized

In [18]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [19]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [20]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-12-25 19:34:34,121] A new study created in memory with name: no-name-0df5d253-a65c-49c7-b787-7b93fcf50286
[I 2025-12-25 19:34:35,228] Trial 0 finished with value: 0.7540889526542324 and parameters: {'classifier': 'RandomForest', 'n_estimators': 290, 'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 6, 'bootstrap': True}. Best is trial 0 with value: 0.7540889526542324.
[I 2025-12-25 19:34:36,381] Trial 1 finished with value: 0.7671130240714171 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 269, 'learning_rate': 0.02694121055766108, 'max_depth': 3, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.7671130240714171.
[I 2025-12-25 19:34:39,168] Trial 2 finished with value: 0.7557548222541048 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 175, 'learning_rate': 0.06843159840836566, 'max_depth': 13, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.7671130240714171.
[I 2025-12-25 

In [21]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)
study.trials_dataframe()

Best trial parameters: {'classifier': 'GradientBoosting', 'n_estimators': 285, 'learning_rate': 0.011176776944023407, 'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 6}
Best trial accuracy: 0.7671608480790691


,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.754089,2025-12-25 19:34:34.122030,2025-12-25 19:34:35.228232,0 days 00:00:01.106202,NaN,True,RandomForest,NaN,NaN,NaN,6.0,6.0,2.0,290.0,COMPLETE
1,1,0.767113,2025-12-25 19:34:35.229368,2025-12-25 19:34:36.381013,0 days 00:00:01.151645,NaN,NaN,GradientBoosting,NaN,NaN,0.026941,3.0,4.0,5.0,269.0,COMPLETE
2,2,0.755755,2025-12-25 19:34:36.382018,2025-12-25 19:34:39.168244,0 days 00:00:02.786226,NaN,NaN,GradientBoosting,NaN,NaN,0.068432,13.0,4.0,6.0,175.0,COMPLETE
3,3,0.760609,2025-12-25 19:34:39.170314,2025-12-25 19:34:39.555761,0 days 00:00:00.385447,NaN,True,RandomForest,NaN,NaN,NaN,12.0,6.0,6.0,61.0,COMPLETE
4,4,0.739455,2025-12-25 19:34:39.557130,2025-12-25 19:34:46.246388,0 days 00:00:06.689258,NaN,NaN,GradientBoosting,NaN,NaN,0.027008,19.0,4.0,4.0,290.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.762259,2025-12-25 19:38:21.712215,2025-12-25 19:38:21.965615,0 days 00:00:00.253400,NaN,NaN,GradientBoosting,NaN,NaN,0.051979,3.0,7.0,4.0,87.0,COMPLETE
96,96,0.755739,2025-12-25 19:38:21.966611,2025-12-25 19:38:22.261774,0 days 00:00:00.295163,NaN,NaN,GradientBoosting,NaN,NaN,0.011430,4.0,7.0,4.0,76.0,COMPLETE
97,97,0.749211,2025-12-25 19:38:22.262770,2025-12-25 19:38:22.683892,0 days 00:00:00.421122,NaN,NaN,GradientBoosting,NaN,NaN,0.030172,4.0,8.0,4.0,90.0,COMPLETE
98,98,0.758999,2025-12-25 19:38:22.684879,2025-12-25 19:38:22.948339,0 days 00:00:00.263460,NaN,NaN,GradientBoosting,NaN,NaN,0.042208,3.0,7.0,5.0,80.0,COMPLETE


In [22]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
GradientBoosting    64
RandomForest        28
SVM                  8
Name: count, dtype: int64

In [23]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.751924
RandomForest        0.751769
SVM                 0.717851
Name: value, dtype: float64

In [24]:
# 1. Optimization History
plot_optimization_history(study).show()

In [25]:
# 3. Slice Plot
plot_slice(study).show()

In [26]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [28]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")

[I 2025-12-25 19:39:46,323] A new study created in memory with name: no-name-1e8a6889-afd7-41db-9da6-a4b7f481dbe1


[0]	train-mlogloss:0.78846	eval-mlogloss:0.77323
[1]	train-mlogloss:0.59518	eval-mlogloss:0.57319
[2]	train-mlogloss:0.45766	eval-mlogloss:0.42983
[3]	train-mlogloss:0.36084	eval-mlogloss:0.32526
[4]	train-mlogloss:0.28730	eval-mlogloss:0.25186
[5]	train-mlogloss:0.23327	eval-mlogloss:0.19409
[6]	train-mlogloss:0.19547	eval-mlogloss:0.15507
[7]	train-mlogloss:0.16243	eval-mlogloss:0.12258
[8]	train-mlogloss:0.13665	eval-mlogloss:0.10172
[9]	train-mlogloss:0.11321	eval-mlogloss:0.08970
[10]	train-mlogloss:0.09795	eval-mlogloss:0.07273
[11]	train-mlogloss:0.08747	eval-mlogloss:0.06070
[12]	train-mlogloss:0.07874	eval-mlogloss:0.05710
[13]	train-mlogloss:0.07261	eval-mlogloss:0.05089
[14]	train-mlogloss:0.06850	eval-mlogloss:0.04534
[15]	train-mlogloss:0.06650	eval-mlogloss:0.04385
[16]	train-mlogloss:0.06364	eval-mlogloss:0.04081
[17]	train-mlogloss:0.06217	eval-mlogloss:0.04197
[18]	train-mlogloss:0.06134	eval-mlogloss:0.04159
[19]	train-mlogloss:0.05989	eval-mlogloss:0.03936
[20]	train

[I 2025-12-25 19:39:46,634] Trial 0 finished with value: 1.0 and parameters: {'lambda': 2.5805257300526665e-06, 'alpha': 1.5301745920164108e-05, 'eta': 0.2546927402087136, 'gamma': 0.007206211850039753, 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.8889776264913811, 'colsample_bytree': 0.9190942806503279}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.87610	eval-mlogloss:0.87063
[1]	train-mlogloss:0.71571	eval-mlogloss:0.69887
[2]	train-mlogloss:0.59147	eval-mlogloss:0.56891
[3]	train-mlogloss:0.49730	eval-mlogloss:0.47237
[4]	train-mlogloss:0.42266	eval-mlogloss:0.39599
[5]	train-mlogloss:0.36242	eval-mlogloss:0.33160
[6]	train-mlogloss:0.31476	eval-mlogloss:0.27957
[7]	train-mlogloss:0.27415	eval-mlogloss:0.23724
[8]	train-mlogloss:0.24275	eval-mlogloss:0.20336
[9]	train-mlogloss:0.21505	eval-mlogloss:0.17695
[10]	train-mlogloss:0.19280	eval-mlogloss:0.15576
[11]	train-mlogloss:0.17572	eval-mlogloss:0.13574
[12]	train-mlogloss:0.16063	eval-mlogloss:0.12197
[13]	train-mlogloss:0.14958	eval-mlogloss:0.11006
[14]	train-mlogloss:0.14204	eval-mlogloss:0.09991
[15]	train-mlogloss:0.13480	eval-mlogloss:0.09069
[16]	train-mlogloss:0.13237	eval-mlogloss:0.08622
[17]	train-mlogloss:0.12995	eval-mlogloss:0.08379
[18]	train-mlogloss:0.12811	eval-mlogloss:0.08105
[19]	train-mlogloss:0.12573	eval-mlogloss:0.07807
[20]	train

[I 2025-12-25 19:39:47,012] Trial 1 finished with value: 1.0 and parameters: {'lambda': 3.281683654192561e-08, 'alpha': 0.0021479203790972083, 'eta': 0.1802221701464783, 'gamma': 0.10744174268939634, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.9444147668009538, 'colsample_bytree': 0.7764765387215209}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.91560	eval-mlogloss:0.91075


[I 2025-12-25 19:39:49,921] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:1.07774	eval-mlogloss:1.07712
[1]	train-mlogloss:1.06701	eval-mlogloss:1.06696
[2]	train-mlogloss:1.05329	eval-mlogloss:1.05401
[3]	train-mlogloss:1.03015	eval-mlogloss:1.02952


[I 2025-12-25 19:39:49,944] Trial 49 pruned. Trial was pruned at iteration 4.


Best trial: {'lambda': 2.5805257300526665e-06, 'alpha': 1.5301745920164108e-05, 'eta': 0.2546927402087136, 'gamma': 0.007206211850039753, 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.8889776264913811, 'colsample_bytree': 0.9190942806503279}
Best accuracy: 1.0


In [29]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()